# Bell States & Entanglement

Here, we prepare two quantum states:
$$ | \psi_{00} \rangle_1 = \hat C_{10} \hat H_1 | 00 \rangle =  \frac{1}{\sqrt{2}} ( | 00 \rangle + | 11 \rangle ) $$
(one of Bell's states) and
$$ | \phi_{00} \rangle_2 = \hat H_1 \hat H_0 | 00 \rangle = \frac{1}{2} ( | 00 \rangle + | 01 \rangle + | 10 \rangle + | 11 \rangle ). $$

Then, we measure $\hat Z_1 \hat Z_0$ and $\hat Z_1 \hat I_0$ in both states. The expected values are
$$ \langle ZZ \rangle_1 = 1 $$
(measure of entanglement) and
$$ \langle ZI \rangle_1 = \langle ZZ \rangle_2 = \langle ZI \rangle_2 = 0. $$

In [3]:
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Pauli
from qiskit_aer.primitives import Estimator as FakeEstimator
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as RealEstimator

service = QiskitRuntimeService()
backend = service.least_busy(simulator=False, operational=True)

In [ ]:
# Prepare state CH|00>

qc1 = QuantumCircuit(2)

qc1.h(0)
qc1.cx(0, 1)

qc1.draw()

┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘

In [6]:
# Optimize for real quantum computer

isa_circuit1 = transpile(qc1, backend=backend, optimization_level=1)
isa_circuit1.draw()

global phase: 3π/4
         ┌─────────┐┌────┐          ┌──────┐┌───┐
q_0 -> 0 ┤ Rz(π/2) ├┤ √X ├──────────┤0     ├┤ X ├
         └┬────────┤├────┤┌────────┐│  Ecr │└───┘
q_1 -> 1 ─┤ Rz(-π) ├┤ √X ├┤ Rz(-π) ├┤1     ├─────
          └────────┘└────┘└────────┘└──────┘

In [7]:
# Prepare state HH|00>

qc2 = QuantumCircuit(2)

qc2.h(0)
qc2.h(1)

qc2.draw()

┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     └───┘

In [8]:
# Optimize for real quantum computer

isa_circuit2 = transpile(qc2, backend=backend, optimization_level=1)
isa_circuit2.draw()

global phase: π/2
         ┌─────────┐┌────┐┌─────────┐
q_0 -> 0 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├
         ├─────────┤├────┤├─────────┤
q_1 -> 1 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├
         └─────────┘└────┘└─────────┘

In [9]:
# Set up observables

ZZ = Pauli('ZZ')
ZI = Pauli('ZI')

In [25]:
# Run in a simulation

fake_estimator = FakeEstimator()

fake_job = fake_estimator.run(
  [qc1, qc1, qc2, qc2],
  [ ZZ,  ZI,  ZZ,  ZI]
)

fake_result = fake_job.result()
print(f'shots = {fake_result.metadata[0]['shots']}')
print()
print(f'<ZZ>_1 = {fake_result.values[0]}')
print(f'<ZI>_1 = {fake_result.values[1]}')
print(f'<ZZ>_2 = {fake_result.values[2]}')
print(f'<ZI>_2 = {fake_result.values[2]}')


shots = 1024

<ZZ>_1 = 1.0
<ZI>_1 = -0.029296875
<ZZ>_2 = -0.021484375
<ZI>_2 = -0.021484375


In [ ]:
# Run in a real quantum computer

real_estimator = RealEstimator(mode=backend)

real_job = real_estimator.run([
  (isa_circuit1, ZZ.apply_layout(isa_circuit1.layout)),
  (isa_circuit1, ZI.apply_layout(isa_circuit1.layout)),
  (isa_circuit2, ZZ.apply_layout(isa_circuit2.layout)),
  (isa_circuit2, ZI.apply_layout(isa_circuit2.layout))
])

print(f"Job ID: {real_job.job_id()}")

Job ID: czs28ptnhqag008sqf30


In [ ]:
# Show result from real quantum computer

real_result = real_job.result()

print(f'shots = {real_result[0].metadata['shots']}')
print()
print(f'<ZZ>_1 = {real_result[0].data.evs}')
print(f'<ZI>_1 = {real_result[1].data.evs}')
print(f'<ZZ>_2 = {real_result[2].data.evs}')
print(f'<ZI>_2 = {real_result[3].data.evs}')

shots = 4096

<ZZ>_1 = 0.9957168052406148
<ZI>_1 = -0.0177449168207024
<ZZ>_2 = 0.013605442176870748
<ZI>_2 = 0.012322858903265557
